In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.models as models
import sys
import math
import matplotlib.pyplot as plt
import statistics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix





In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
Resnet50_model=torch.jit.load("../input/models/Resnet50.pt")
Densenet121_model=torch.jit.load("../input/models/Densenet121.pt")
efficientnet_model=torch.jit.load("../input/models/EfficientNet.pt")


In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
#Dataloader

#Path for training and testing directory
train_path='../input/final-data-with-valid-folder/Final_DataSet_With_Train_Test_And_Validation/train'
valid_path='../input/final-data-with-valid-folder/Final_DataSet_With_Train_Test_And_Validation/valid'
test_path='../input/final-data-with-valid-folder/Final_DataSet_With_Train_Test_And_Validation/test'


train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=8, shuffle=False
)
valid_loader=DataLoader(
    torchvision.datasets.ImageFolder(valid_path,transform=transformer),
    batch_size=8, shuffle=False
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=8, shuffle=False
)


In [6]:
train_loader.dataset

In [7]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [8]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [9]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


In [10]:
# test_loader = DeviceDataLoader(test_loader, device)
# Resnet50_model.
# # result = evaluate(Densenet121_model, test_loader)
# result


In [11]:
nb_classes = 2

confusion_matrix_densenet = torch.zeros(nb_classes, nb_classes)
confusion_matrix_resnet = torch.zeros(nb_classes, nb_classes)
confusion_matrix_efficientnet = torch.zeros(nb_classes, nb_classes)
confusion_matrix_bagging = torch.zeros(nb_classes, nb_classes)

prediction_densenet=[]
prediction_resnet=[]
prediction_efficientnet=[]
models_predictions=[]
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs_densenet = Densenet121_model(inputs)
        outputs_resnet = Resnet50_model(inputs)
        outputs_efficientnet = efficientnet_model(inputs)
        d, preds_densenet = torch.max(outputs_densenet, 1)
        r,preds_resnet=torch.max(outputs_resnet,1)
        e,preds_efficientnet=torch.max(outputs_efficientnet,1)
        alist=[]
        prediction_bagging=[]
        alist.append(preds_densenet.tolist())
        alist.append(preds_resnet.tolist())
        alist.append(preds_efficientnet.tolist())
        
        for item in preds_densenet.tolist():
            prediction_densenet.append(item)

            
        for item in preds_resnet.tolist():
            prediction_resnet.append(item)
    
        
        for item in preds_efficientnet.tolist():
            prediction_efficientnet.append(item)
    
        
       
            
        for i in range(len(alist[0])):
            lst=[]
            lst.append(alist[0][i])
            lst.append(alist[1][i])
            lst.append(alist[2][i])
            prediction_bagging.append(statistics.mode(lst))
        
        preds_bagging=torch.tensor(prediction_bagging)
        preds_bagging=preds_bagging.to(device)
            
            
        
            
        for t, p in zip(classes.view(-1), preds_densenet.view(-1)):
                confusion_matrix_densenet[t.long(), p.long()] += 1
                
        for t, p in zip(classes.view(-1), preds_resnet.view(-1)):
                confusion_matrix_resnet[t.long(), p.long()] += 1
        
        for t, p in zip(classes.view(-1), preds_efficientnet.view(-1)):
                confusion_matrix_efficientnet[t.long(), p.long()] += 1
        
        for t, p in zip(classes.view(-1), preds_bagging.view(-1)):
                confusion_matrix_bagging[t.long(), p.long()] += 1

models_predictions.append(prediction_densenet)
models_predictions.append(prediction_resnet)
models_predictions.append(prediction_efficientnet)

       

In [12]:
def metrics(confusion_matrix):
    TP = confusion_matrix[0][0]
    FN = confusion_matrix[1][0]
    FP = confusion_matrix[0][1]
    TN = confusion_matrix[1][1]
    
    accuracy=float((TP+TN)/(TP+TN+FP+FN))
    f1_score=float(2*TP/(2*TP+FN+FP))
    precision=float(TP/(TP+FP))
    recall=float(TP/(TP+FN))
    
    return accuracy,f1_score,precision,recall
    

In [13]:
accuracy,f1_score,precision,recall=metrics(confusion_matrix_bagging)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

In [14]:
def ensemble_bagging_stacking(confusion_matrix,model_name):
    accuracy,f1_score,precision,recall=metrics(confusion_matrix)
    print("*************************************************************************************")
    print(f"Accuracy of {model_name} :", accuracy)
    print(f"f1 Score of {model_name} :",f1_score)
    print(f"Precision of {model_name} :",precision)
    print(f"Recall of {model_name} :",recall)
    print("---------------------------------------------------------------------------------------")
    print("\n")
    

In [15]:
def create_stacked_dataset(members,test_loader):
    stackX=None
    confusion_matrix=[]
    for member in members:
        prediction=[]
        label=[]
        c_matrix=torch.zeros(nb_classes, nb_classes)
       
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            c=classes
            outputs = member(inputs)
            classes=classes.cpu().detach().numpy()

            a, preds= torch.max(outputs, 1)
            a=a.cpu().detach().numpy()
            for item in a.tolist():
                prediction.append(item)
            for item in classes.tolist():
                label.append(item)
                
            
            for t, p in zip(c.view(-1), preds.view(-1)):
                c_matrix[t.long(), p.long()] += 1
                
        
            
        confusion_matrix.append(c_matrix)
            
            
        
        if stackX is None:
            stackX=prediction
        else:
            stackX=np.dstack((stackX,prediction))
    
    print(stackX.shape)
    stackX=stackX.reshape(stackX.shape[1],stackX.shape[0],3)
            
    # flatten predictions to [rows, members * probabilities]
    stackX=stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
#     return stackX,label   
    return stackX,label,confusion_matrix

In [16]:
members=[]
members.append(Densenet121_model)
members.append(Resnet50_model)
members.append(efficientnet_model)
def stacking(members,train_loader,test_loader):
    stackedX,label,c_matrix=create_stacked_dataset(members,train_loader)
    meta_model=SVC()
    meta_model.fit(stackedX,label)
    test_X,test_label,test_c_matrix=create_stacked_dataset(members,test_loader)
    meta_model_pred=meta_model.predict(test_X)
    accuracy=accuracy_score(test_label,meta_model_pred)
    c_matrix_meta=confusion_matrix(test_label,meta_model_pred)
    
   
    
    return ("Acuracy_Score_Stacking :",accuracy),test_c_matrix,c_matrix_meta
    
    
    


In [17]:
a,test_c_matrix,c_matrix_meta=stacking(members,train_loader,test_loader)


In [18]:
c_matrix_densenet=test_c_matrix[0]
c_matrix_resnet=test_c_matrix[1]
c_matrix_efficientnet=test_c_matrix[2]


In [19]:
#results 
ensemble_bagging_stacking(confusion_matrix_densenet,"densenet121")
ensemble_bagging_stacking(confusion_matrix_resnet,'resnet50')
ensemble_bagging_stacking(confusion_matrix_efficientnet,'efficientnetB0')
ensemble_bagging_stacking(confusion_matrix_bagging,'Bagging')
ensemble_bagging_stacking(c_matrix_meta,"Stacking")

